In [1]:
import unicodedata
import re
import json
import os
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import requests
import nltk
from nltk.corpus import stopwords
from time import strftime

In [210]:
int(max_page)

6

In [48]:
def get_all_repository_urls(url): # repository url
    # Get max page.
    response = get(url)
    soup = BeautifulSoup(response.text)
    print('Finding max page for repositories...')
    max_page = int(soup.find('div', role='navigation').text[-6])
    print(f'Max page found: {max_page}')
    page = 1
    repository_links = []
    print('Starting loop...')
    for n in range(max_page):
        print(n+1, 'iteration')
        print(f'Pulling data from {url}')
        # Reset soup.
        response = get(url)
        soup = BeautifulSoup(response.text)
        # Get all the repositories from the page.
        repositories = soup.find_all('a', itemprop='name codeRepository') 
        print('Fetching links for repositories...')
        for repo in repositories:
            repository_links.append(repo.get('href'))
        git = 'https://github.com/'
        next_page = soup.find('a', class_='next_page').get('href')[:-1]
        ## Use this line of code to get the url for the next page.
        if page <= 4:
            url = git + next_page + str(page + 1)
            page += 1
        else:
            return repository_links


In [49]:
repository_links = get_all_repository_urls('https://github.com/orgs/apple/repositories')

Finding max page for repositories...
Max page found: 6
Starting loop...
1 iteration
Pulling data from https://github.com/orgs/apple/repositories
Fetching links for repositories...
2 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=2
Fetching links for repositories...
3 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=3
Fetching links for repositories...
4 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=4
Fetching links for repositories...
5 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=5
Fetching links for repositories...


In [51]:
len(repository_links)

150

In [222]:
def get_all_repository_readme(url): # repository url
    response = get(url)
    soup = BeautifulSoup(response.text)
    repositories = soup.find_all('div', class_='org-repos')[0]
    repository_title = repositories.find_all('a', itemprop ='name codeRepository')
    title_list = [title.text.strip() for title in repository_title]
    # Get url for each repository.
    repository_links = []
    for titles in repository_title:
        repository_links.append(titles.get('href'))
    readme_list = []
    git = 'https://github.com/'
    for link in repository_links:
        repo_response = get(git + link)
        repo_soup = BeautifulSoup(repo_response.text)
        readme = repo_soup.find_all('article', itemprop='text')[0].text
        readme = readme.replace('\n',' ').replace('  ', ' ').replace('\'', '')
        readme_list.append(readme)
    dictionary = dict(zip(title_list, readme_list))
    df = pd.DataFrame(list(dictionary.items()), columns=['repo_title','readme'])
    return df

In [2]:
url = 'https://github.com/orgs/apple/repositories'
response = get(url)

In [4]:
soup = BeautifulSoup(response.text)

In [157]:
repositories = soup.find_all('div', class_='org-repos')[0]

In [158]:
repositories

<div class="org-repos repo-list">
<h2 class="f4 text-normal d-md-none pb-3">Repositories</h2>
<div class="Box" data-view-component="true">
<ul>
<li class="Box-row" data-view-component="true"> <div class="public fork d-block py-0 border-0" data-view-component="true" itemprop="owns" itemscope="itemscope" itemtype="http://schema.org/Code">
<div class="d-flex flex-justify-between" data-view-component="true">
<div class="flex-auto" data-view-component="true">
<h3 class="wb-break-all" data-view-component="true">
<a class="d-inline-block" data-hovercard-type="repository" data-hovercard-url="/apple/llvm-project/hovercard" data-view-component="true" href="/apple/llvm-project" itemprop="name codeRepository">
            llvm-project
</a>
<span class="Label Label--secondary v-align-middle ml-1 mb-1" data-view-component="true" title="Label: Public">
            Public
</span></h3>
<span class="color-fg-muted mb-1 f6" data-view-component="true">
            Forked from
            <a class="Link--m

In [166]:
repositories.find_all('a', itemprop ='name codeRepository')

[<a class="d-inline-block" data-hovercard-type="repository" data-hovercard-url="/apple/llvm-project/hovercard" data-view-component="true" href="/apple/llvm-project" itemprop="name codeRepository">
             llvm-project
 </a>,
 <a class="d-inline-block" data-hovercard-type="repository" data-hovercard-url="/apple/swift/hovercard" data-view-component="true" href="/apple/swift" itemprop="name codeRepository">
             swift
 </a>,
 <a class="d-inline-block" data-hovercard-type="repository" data-hovercard-url="/apple/swift-experimental-string-processing/hovercard" data-view-component="true" href="/apple/swift-experimental-string-processing" itemprop="name codeRepository">
             swift-experimental-string-processing
 </a>,
 <a class="d-inline-block" data-hovercard-type="repository" data-hovercard-url="/apple/swift-docker/hovercard" data-view-component="true" href="/apple/swift-docker" itemprop="name codeRepository">
             swift-docker
 </a>,
 <a class="d-inline-block" da

In [167]:
# List of repository titles. Will use to iterate through.
repository_title = repositories.find_all('a', itemprop='name codeRepository')

In [190]:
repository_title

[<a class="d-inline-block" data-hovercard-type="repository" data-hovercard-url="/apple/llvm-project/hovercard" data-view-component="true" href="/apple/llvm-project" itemprop="name codeRepository">
             llvm-project
 </a>,
 <a class="d-inline-block" data-hovercard-type="repository" data-hovercard-url="/apple/swift/hovercard" data-view-component="true" href="/apple/swift" itemprop="name codeRepository">
             swift
 </a>,
 <a class="d-inline-block" data-hovercard-type="repository" data-hovercard-url="/apple/swift-experimental-string-processing/hovercard" data-view-component="true" href="/apple/swift-experimental-string-processing" itemprop="name codeRepository">
             swift-experimental-string-processing
 </a>,
 <a class="d-inline-block" data-hovercard-type="repository" data-hovercard-url="/apple/swift-docker/hovercard" data-view-component="true" href="/apple/swift-docker" itemprop="name codeRepository">
             swift-docker
 </a>,
 <a class="d-inline-block" da

In [202]:
title_list = [title.text.strip() for title in repository_title]

In [203]:
title_list

['llvm-project',
 'swift',
 'swift-experimental-string-processing',
 'swift-docker',
 'swift-docc-symbolkit',
 'swift-package-manager',
 'swift-protobuf',
 'foundationdb',
 'swift-installer-scripts',
 'swift-llbuild2',
 'swift-docc-render',
 'swift-corelibs-foundation',
 'swift-evolution',
 'servicetalk',
 'swift-docc',
 'swift-nio-ssh',
 'password-manager-resources',
 'swift-nio',
 'ml-hypersim',
 'swift-collections',
 'sourcekit-lsp',
 'swift-markdown',
 'swift-docc-render-artifact',
 'swift-lmdb',
 'swift-tools-support-core',
 'swift-driver',
 'swift-syntax',
 'indexstore-db',
 'swift-stress-tester',
 'swift-xcode-playground-support']

In [173]:
repository_title[2].text.strip()

'swift-experimental-string-processing'

In [178]:
for titles in repository_title:
    repository_links.append(titles.get('href'))

In [16]:
# Get url for each repository.
links = repositories.find_all('a', itemprop = 'name codeRepository')

In [175]:
repository_links = []

In [23]:
for link in links:
    repository_links.append(link.get('href'))

In [192]:
repository_links

['/apple/llvm-project',
 '/apple/swift',
 '/apple/swift-experimental-string-processing',
 '/apple/swift-docker',
 '/apple/swift-docc-symbolkit',
 '/apple/swift-package-manager',
 '/apple/swift-protobuf',
 '/apple/foundationdb',
 '/apple/swift-installer-scripts',
 '/apple/swift-llbuild2',
 '/apple/swift-docc-render',
 '/apple/swift-corelibs-foundation',
 '/apple/swift-evolution',
 '/apple/servicetalk',
 '/apple/swift-docc',
 '/apple/swift-nio-ssh',
 '/apple/password-manager-resources',
 '/apple/swift-nio',
 '/apple/ml-hypersim',
 '/apple/swift-collections',
 '/apple/sourcekit-lsp',
 '/apple/swift-markdown',
 '/apple/swift-docc-render-artifact',
 '/apple/swift-lmdb',
 '/apple/swift-tools-support-core',
 '/apple/swift-driver',
 '/apple/swift-syntax',
 '/apple/indexstore-db',
 '/apple/swift-stress-tester',
 '/apple/swift-xcode-playground-support']

In [27]:
repository_links[0]

'/apple/llvm-project'

In [31]:
git = 'https://github.com/'

In [32]:
repo_response = get(git + repository_links[0])

In [36]:
repo_response

<Response [200]>

In [37]:
repo_soup = BeautifulSoup(repo_response.text)

In [104]:
repo_soup.find_all('article', itemprop='text')[0].text

'Apple\'s fork of llvm-project\nThis is Apple\'s fork of llvm-project.  For more information on Apple\'s\nbranching scheme, please see\napple-docs/AppleBranchingScheme.md.\nThe LLVM project\'s main README follows.\nThe LLVM Compiler Infrastructure\nThis directory and its sub-directories contain source code for LLVM,\na toolkit for the construction of highly optimized compilers,\noptimizers, and run-time environments.\nThe README briefly describes how to get started with building LLVM.\nFor more information on how to contribute to the LLVM project, please\ntake a look at the\nContributing to LLVM guide.\nGetting Started with the LLVM System\nTaken from https://llvm.org/docs/GettingStarted.html.\nOverview\nWelcome to the LLVM project!\nThe LLVM project has multiple components. The core of the project is\nitself called "LLVM". This contains all of the tools, libraries, and header\nfiles needed to process intermediate representations and convert them into\nobject files.  Tools include an a

In [128]:
readme = repo_soup.find_all('article', itemprop='text')[0].text

In [129]:
readme = readme.replace('\n',' ').replace('  ', ' ').replace('\'', '')

In [134]:
readme

'Apples fork of llvm-project This is Apples fork of llvm-project. For more information on Apples branching scheme, please see apple-docs/AppleBranchingScheme.md. The LLVM projects main README follows. The LLVM Compiler Infrastructure This directory and its sub-directories contain source code for LLVM, a toolkit for the construction of highly optimized compilers, optimizers, and run-time environments. The README briefly describes how to get started with building LLVM. For more information on how to contribute to the LLVM project, please take a look at the Contributing to LLVM guide. Getting Started with the LLVM System Taken from https://llvm.org/docs/GettingStarted.html. Overview Welcome to the LLVM project! The LLVM project has multiple components. The core of the project is itself called "LLVM". This contains all of the tools, libraries, and header files needed to process intermediate representations and convert them into object files. Tools include an assembler, disassembler, bitcod

In [97]:
next_page = soup.find('a', class_='next_page').get('href')[:-1]

In [230]:
page = 1

In [232]:
# Use this line of code to set the next url.
next_page + str(page + 1)

'/orgs/apple/repositories?page=2'

In [227]:
next_page

'/orgs/apple/repositories?page='

In [101]:
git + next_page + max_page

'https://github.com//orgs/apple/repositories?page=6'

In [100]:
max_page = soup.find('div', role='navigation').text[-6]

In [102]:
type(max_page)

str

In [193]:
readme_list = []
git = 'https://github.com/'
for link in repository_links:
    repo_response = get(git + link)
    repo_soup = BeautifulSoup(repo_response.text)
    readme = repo_soup.find_all('article', itemprop='text')[0].text
    readme = readme.replace('\n',' ').replace('  ', ' ').replace('\'', '')
    readme_list.append(readme)

In [196]:
len(readme_list)

30

In [204]:
dictionary = dict(zip(title_list, readme_list))

In [221]:
pd.DataFrame(list(dictionary.items()), columns=['repo_title','readme'])

,repo_title,readme
0,llvm-project,Apples fork of llvm-project This is Apples for...
1,swift,Swift Programming Language Architecture mai...
2,swift-experimental-string-processing,Declarative String Processing for Swift An ear...
3,swift-docker,swift-docker Docker images for Swift. You can ...
4,swift-docc-symbolkit,SymbolKit The specification and reference mode...
5,swift-package-manager,Swift Package Manager Project The Swift Packag...
6,swift-protobuf,Swift Protobuf Welcome to Swift Protobuf! App...
7,foundationdb,FoundationDB is a distributed database design...
8,swift-installer-scripts,Swift Installer Scripts This repository contai...
9,swift-llbuild2,"llbuild2 llbuild2 is an experimental, Swift na..."


In [223]:
b = get_all_repository_readme('https://github.com/orgs/apple/repositories')

In [353]:
b

,repo_title,readme
0,llvm-project,Apples fork of llvm-project This is Apples for...
1,swift,Swift Programming Language Architecture mai...
2,swift-driver,Swift Compiler Driver Swifts compiler driver i...
3,swift-corelibs-libdispatch,Grand Central Dispatch Grand Central Dispatch ...
4,foundationdb,FoundationDB is a distributed database design...
5,swift-package-manager,Swift Package Manager Project The Swift Packag...
6,swift-docc-symbolkit,SymbolKit The specification and reference mode...
7,swift-experimental-string-processing,Declarative String Processing for Swift An ear...
8,swift-docker,swift-docker Docker images for Swift. You can ...
9,swift-protobuf,Swift Protobuf Welcome to Swift Protobuf! App...


In [34]:
def get_all_repository_urls(url): # repository url
    # Get max page.
    response = get(url)
    soup = BeautifulSoup(response.text)
    print('Finding max page for repositories...')
    max_page = int(soup.find('div', role='navigation').text[-6])
    print(f'Max page found: {max_page}')
    page = 1
    title_list = []
    readme_list = []
    repository_links = []
    print('Starting loop...')
    for n in range(max_page):
        print(n+1, 'iteration')
        print(f'Pulling data from {url}')
        # Reset soup.
        response = get(url)
        soup = BeautifulSoup(response.text)
        # Get all the repositories from the page.
        repositories = soup.find_all('a', itemprop='name codeRepository')
        # Create a list of the repository titles.
        repository_titles = [repo.text.strip() for repo in repositories]
        repository_titles = [e for e in repository_titles if e not in ('darwin-libpthread')]
        print("Appending titles to 'title_list'...")
        for title in repository_titles:
            title_list.append(title)
        print('Repository titles appended.')     
        print('Fetching links for repositories...')
        for repo in repositories:
            repository_links.append(repo.get('href'))
        ## Create a list of the readme files to add to a dataframe
        git = 'https://github.com/'
        #print('Fetching README files from each repository...')
        #for link in repository_links:
        #    print(f'Fetching from {link}...')
        #    # Append github to beginning of repository url
        #    repo_response = get(git + link)
        #    repo_soup = BeautifulSoup(repo_response.text)
        #    # Get the readme from the page.
        #    readme = repo_soup.find('div', id='readme').text.strip().replace('\n', ' ')
        #    readme_list.append(readme)
        #print('README files obtained.')
        next_page = soup.find('a', class_='next_page').get('href')[:-1]
        ## Use this line of code to get the url for the next page.
        if page <= 5:
            url = git + next_page + str(page + 1)
            page += 1
        else:
            return repository_links
        

In [35]:
list_of_repositories = get_all_repository_urls('https://github.com/orgs/apple/repositories')

Finding max page for repositories...
Max page found: 6
Starting loop...
1 iteration
Pulling data from https://github.com/orgs/apple/repositories
Appending titles to 'title_list'...
Repository titles appended.
Fetching links for repositories...
2 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=2
Appending titles to 'title_list'...
Repository titles appended.
Fetching links for repositories...
3 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=3
Appending titles to 'title_list'...
Repository titles appended.
Fetching links for repositories...
4 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=4
Appending titles to 'title_list'...
Repository titles appended.
Fetching links for repositories...
5 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=5
Appending titles to 'title_list'...
Repository titles appended.
Fetching links for repositories...
6 iteration
Pulling data from https:

AttributeError: 'NoneType' object has no attribute 'get'

In [37]:
list_of_repositories

180

In [10]:
response = get('https://github.com/orgs/apple/repositories')
soup = BeautifulSoup(response.text)

In [13]:
repositories = soup.find_all('a', itemprop='name codeRepository')

In [23]:
list_o_repo = []
for repo in repositories:
    list_o_repo.append(repo.get('href'))

In [24]:
list_o_repo

['/apple/llvm-project',
 '/apple/swift',
 '/apple/foundationdb',
 '/apple/swift-docc',
 '/apple/swift-protobuf',
 '/apple/swift-llbuild',
 '/apple/swift-syntax',
 '/apple/swift-package-manager',
 '/apple/swift-source-compat-suite',
 '/apple/sourcekit-lsp',
 '/apple/swift-markdown',
 '/apple/swift-docc-symbolkit',
 '/apple/swift-docc-render-artifact',
 '/apple/swift-lmdb',
 '/apple/swift-tools-support-core',
 '/apple/swift-driver',
 '/apple/indexstore-db',
 '/apple/swift-stress-tester',
 '/apple/swift-xcode-playground-support',
 '/apple/swift-integration-tests',
 '/apple/swift-cmark',
 '/apple/swift-corelibs-libdispatch',
 '/apple/swift-corelibs-foundation',
 '/apple/swift-corelibs-xctest',
 '/apple/swift-experimental-string-processing',
 '/apple/swift-docker',
 '/apple/swift-installer-scripts',
 '/apple/swift-llbuild2',
 '/apple/swift-docc-render',
 '/apple/swift-evolution']

In [44]:
def get_all_repository_urls(url): # repository url
    # Get max page.
    response = get(url)
    soup = BeautifulSoup(response.text)
    print('Finding max page for repositories...')
    max_page = int(soup.find('div', role='navigation').text[-6])
    print(f'Max page found: {max_page}')
    page = 1
    title_list = []
    readme_list = []
    repository_links = []
    print('Starting loop...')
    for n in range(max_page):
        print(n+1, 'iteration')
        print(f'Pulling data from {url}')
        # Reset soup.
        response = get(url)
        soup = BeautifulSoup(response.text)
        # Get all the repositories from the page.
        repositories = soup.find_all('a', itemprop='name codeRepository')
        # Create a list of the repository titles.
        repository_titles = [repo.text.strip() for repo in repositories]
        repository_titles = [e for e in repository_titles if e not in ('darwin-libpthread')]
        print("Appending titles to 'title_list'...")
        for title in repository_titles:
            title_list.append(title.text.strip())
        print('Repository titles appended.')
        # Get url for each repository.
        repository_links = []
        print('Fetching links for repositories...')
        for titles in repository_titles:
            repository_links.append(titles.get('href'))
        # Create a list of the readme files to add to a dataframe
        git = 'https://github.com/'
        #print('Fetching README files from each repository...')
        #for link in repository_links:
        #    print(f'Fetching from {link}...')
        #    # Append github to beginning of repository url
        #    repo_response = get(git + link)
        #    repo_soup = BeautifulSoup(repo_response.text)
        #    # Get the readme from the page.
        #    readme = repo_soup.find('div', id='readme').text.strip().replace('\n', ' ')
        #    readme_list.append(readme)
        #print('README files obtained.')
        next_page = soup.find('a', class_='next_page').get('href')[:-1]


In [45]:
repository_links = get_all_repository_urls('https://github.com/orgs/apple/repositories')

Finding max page for repositories...
Max page found: 6
Starting loop...
1 iteration
Pulling data from https://github.com/orgs/apple/repositories
Appending titles to 'title_list'...


AttributeError: 'str' object has no attribute 'text'

In [9]:
repository_link = get_all_repository_urls('https://github.com/orgs/apple/repositories')

Finding max page for repositories...
Max page found: 6
Starting loop...
1 iteration
Pulling data from https://github.com/orgs/apple/repositories
Appending titles to 'title_list'...
Repository titles appended.
Fetching links for repositories...


AttributeError: 'str' object has no attribute 'get'

In [4]:
title_list

NameError: name 'title_list' is not defined

In [ ]:
# ccs-pyopendirectory, darwin-libplatform

In [74]:
len(readme_list)

147